<a href="https://colab.research.google.com/github/Marzieh777/Fraud-Detection-Crime-Rating/blob/master/fraud_detection_XGB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#importing necessary packages
import pandas as pd
import numpy as np
import gzip
from sklearn.metrics import (confusion_matrix, precision_recall_curve, auc,accuracy_score,
                             roc_curve, recall_score, classification_report, f1_score,
                             precision_recall_fscore_support, roc_auc_score, average_precision_score)
from google.colab import drive
from sklearn.model_selection import train_test_split

import xgboost as xgb
import matplotlib

drive.mount("/content/drive", force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


# Load data and create a panadas dataframe

In [2]:
path_data= '/content/drive/My Drive/Colab Notebooks/fraud_prep.csv.gz'
fraud_df = pd.read_csv(path_data, compression='gzip', header=0, sep=',')
print(fraud_df.head())

   Unnamed: 0  Time  ...                                        V30      V31
0           0   0.0  ...                                        NaN  Suffolk
1           1   0.0  ...  JPMorgan Chase Bank, National Association   Nassau
2           2   1.0  ...                               TD Bank N.A.     Erie
3           3   1.0  ...                             Santander Bank   Ulster
4           4   2.0  ...                Chemung Canal Trust Company   Oswego

[5 rows x 35 columns]


##The data set has NaN values in categorical and nmerical features, the following function helps to fill NaN values

In [3]:
#NaN values for categorical and numerical:
from sklearn.base import TransformerMixin

class DataFrameImputer(TransformerMixin):

    def __init__(self):
        """Impute missing values.

        Columns of dtype object are imputed with the most frequent value 
        in column.

        Columns of other types are imputed with mean of column.

        """
    def fit(self, X, y=None):

        self.fill = pd.Series([X[c].value_counts().index[0]
            if X[c].dtype == np.dtype('O') else X[c].mean() for c in X],
            index=X.columns)

        return self

    def transform(self, X, y=None):
        return X.fillna(self.fill)

fraud_df_clean = DataFrameImputer().fit_transform(fraud_df)
print(len(fraud_df_clean))


284807


#Exploring data, numerical and categorical features

In [4]:
CATEGORICAL_INPUT_FEATURES = ['V{}'.format(i) for i in range(30, 32)]
ALL_FEATURES = list(fraud_df_clean)
INPUT_FEATURES = ALL_FEATURES.copy()
INPUT_FEATURES.remove('Class') #target
INPUT_FEATURES.remove('V29') #seems not important
print(INPUT_FEATURES)
NUMERICAL_INPUT_FEATURES = [feature for feature in INPUT_FEATURES if feature not in CATEGORICAL_INPUT_FEATURES]
print(NUMERICAL_INPUT_FEATURES)
print(CATEGORICAL_INPUT_FEATURES)
# get number of frauds and non-frauds
fraud_df_clean.Class.value_counts()

['Unnamed: 0', 'Time', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10', 'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20', 'V21', 'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28', 'Amount', 'V30', 'V31']
['Unnamed: 0', 'Time', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10', 'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20', 'V21', 'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28', 'Amount']
['V30', 'V31']


0    284315
1       492
Name: Class, dtype: int64

# Split data nito Test and Train 

In [6]:
#set a seed to generate a fix random choice
SEED = 1333 
#splitting data into train and test:
df_train, df_test = train_test_split(fraud_df_clean, test_size=0.3, random_state=SEED)
#explore the number normal and fraud data in train and test
print('#of frauds in train : ',len(df_train[df_train.Class==1]))
print('# of normals in train: ',len(df_train[df_train.Class==0]))
print('#of frauds in test: ', len(df_test[df_test.Class==1]))
print('#of frauds in test: ',len(df_test[df_test.Class==0]))

#get features
X_train= df_train.drop('Class',axis =1)
X_test = df_test.drop('Class',axis =1)

# get labels
y_train =df_train.Class
y_test =df_test.Class
print(len(y_train))
print(len(X_test))



#of frauds in train :  341
# of normals in train:  199023
#of frauds in test:  151
#of frauds in test:  85292
199364
85443


#Preprocessing categorical and numerical data
* Encode categorical features V30, V31.
* Cooncatenating encoded categorical and numerical data


In [7]:
# encode categorical attributes into a binary one-hot encoded representation in test data and train data
def encode_cat_features(cat_cols, train_data,test_data):
  train_encoded = pd.get_dummies(train_data[cat_cols], prefix_sep="__", columns=cat_cols)
  encoded_columns = list(train_encoded.columns[:])
  print(encoded_columns)
  cat_dummies = [col for col in train_encoded if "__" in col and col.split("__")[0] in cat_cols]
  test_encoded = pd.get_dummies(test_data[cat_cols], prefix_sep="__", columns=cat_cols)
  # Remove additional columns
  for col in test_encoded.columns:
    if ("__" in col) and (col.split("__")[0] in cat_cols) and col not in cat_dummies:
      print("Removing additional feature {}".format(col))
      test_encoded.drop(col, axis=1, inplace=True)
  #add missing cols
  for col in cat_dummies:
    if col not in test_encoded.columns:
      print("Adding missing feature {}".format(col))
      test_processed[col] = 0
  test_encoded = test_encoded[encoded_columns]
  return (train_encoded,test_encoded)
#get encoded categorical data
X_train_cat, X_test_cat = encode_cat_features(CATEGORICAL_INPUT_FEATURES,X_train, X_test)
#get numerical data
X_train_num = X_train[NUMERICAL_INPUT_FEATURES]
X_test_num = X_test[NUMERICAL_INPUT_FEATURES]
# concatenating numerical and categorical
df_train= pd.concat([X_train_cat,X_train_num, y_train], axis = 1)
df_test = pd.concat([X_test_cat, X_test_num, y_test], axis = 1)

#we can sample train and test data by fraction of data
#we choose frac = 1 for the last run
df_train_rd = df_train.sample(frac =1)
df_test_rd= df_test.sample(frac = 1)

print(len(df_train_rd))
print(len(df_test_rd))

['V30__Abacus Federal Savings Bank', 'V30__Access Federal Credit Union', 'V30__Adirondack Bank', 'V30__Adirondack Trust Company, The', 'V30__Advantage Federal C.U.', 'V30__Alco Federal Credit Union', 'V30__Alden State Bank', 'V30__Alma Bank', 'V30__Alternatives Federal Credit Union', 'V30__Amalgamated Bank', 'V30__Amerasia Bank', 'V30__AmeriCU Credit Union', 'V30__American Broadcast Employees Federal Credit Union', 'V30__Apple Bank For Savings', 'V30__Atlas Bank', 'V30__Ballston Spa National Bank', 'V30__Banco Popular North America', 'V30__Bank of Akron', 'V30__Bank of America N.A. GA1-006-15-40', 'V30__Bank of Castile, The', 'V30__Bank of Cattaraugus', 'V30__Bank of Holland', 'V30__Bank of Hope', 'V30__Bank of Millbrook', 'V30__Bank of Richmondville', 'V30__Bank of Utica', 'V30__BankUnited, NA', 'V30__Bay Ridge Federal Credit Union', 'V30__Berkshire Bank (Pittsfield, MA)', 'V30__Berkshire Bank, The', 'V30__Bethpage Federal Credit Union', 'V30__Bridgehampton National Bank, The', 'V30__

# Deal with the imbalance data
* we use SMOTE technic to balance data (Synthetic Minority Over-sampling Technique )

In [8]:
from imblearn.over_sampling import SMOTE
# SMOTE
X_train_rd =df_train_rd.drop('Class',axis = 1)
y_train_rd=df_train_rd.Class.values
sm = SMOTE(random_state=42)
X_train_res, y_train_res = sm.fit_sample(X_train_rd, y_train_rd)
#no over sampling on test data
X_test_rd =df_test_rd.drop('Class',axis = 1)
y_test_rd=df_test_rd.Class.values
cols = X_test_rd.columns
# converting trai data to a data frame with the original column names
X_train_res = pd.DataFrame(X_train_res, columns= cols)

/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


#Performing XGboost Classifier

In [10]:
#Implementing XG-boost #max_depth= 10
clf = xgb.XGBClassifier(missing=9999999999,
                max_depth = 10,
                n_estimators=1000,
                learning_rate=0.1,
                nthread=4,
                subsample=1.0,
                colsample_bytree=0.5,
                min_child_weight = 3,
                seed=1301)

#fit classifier with eval_metric=AUC_PR, early stopping if the result does not improve after 30 consecutive iteration
clf.fit(X_train_res, y_train_res, early_stopping_rounds=30, eval_metric="aucpr",
        eval_set=[(X_train_res, y_train_res),(X_test_rd, y_test_rd)])

[0]	validation_0-aucpr:0.994923	validation_1-aucpr:0.598287
Multiple eval metrics have been passed: 'validation_1-aucpr' will be used for early stopping.

Will train until validation_1-aucpr hasn't improved in 30 rounds.
[1]	validation_0-aucpr:0.998882	validation_1-aucpr:0.666558
[2]	validation_0-aucpr:0.998962	validation_1-aucpr:0.677061
[3]	validation_0-aucpr:0.999371	validation_1-aucpr:0.697721
[4]	validation_0-aucpr:0.999456	validation_1-aucpr:0.705365
[5]	validation_0-aucpr:0.999516	validation_1-aucpr:0.704929
[6]	validation_0-aucpr:0.999645	validation_1-aucpr:0.713109
[7]	validation_0-aucpr:0.999727	validation_1-aucpr:0.720772
[8]	validation_0-aucpr:0.999762	validation_1-aucpr:0.766784
[9]	validation_0-aucpr:0.999765	validation_1-aucpr:0.759941
[10]	validation_0-aucpr:0.999791	validation_1-aucpr:0.760337
[11]	validation_0-aucpr:0.999799	validation_1-aucpr:0.761962
[12]	validation_0-aucpr:0.9998	validation_1-aucpr:0.761076
[13]	validation_0-aucpr:0.999801	validation_1-aucpr:0.7601

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.5, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=10,
              min_child_weight=3, missing=9999999999, n_estimators=1000,
              n_jobs=1, nthread=4, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=1301,
              silent=None, subsample=1.0, verbosity=1)

#Display results

In [11]:
#get AUC and AUC_PR on test data
print('Overall AUC TEST:', roc_auc_score(y_test_rd, clf.predict(X_test_rd, ntree_limit=clf.best_iteration))) 	
print('Overall AUC PR TEST:', average_precision_score(y_test_rd, clf.predict(X_test_rd, ntree_limit=clf.best_iteration))) 
# get precision and recall for each class
print (classification_report(y_test_rd, clf.predict(X_test_rd)))

Overall AUC TEST: 0.9171012987561546
Overall AUC PR TEST: 0.7204232286656621
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     85292
           1       0.87      0.83      0.85       151

    accuracy                           1.00     85443
   macro avg       0.93      0.92      0.93     85443
weighted avg       1.00      1.00      1.00     85443



#Plotting AUC (Area under curve)

In [0]:
y_pred =
fpr, tpr, thresholds = roc_curve(y_test_rd, y_pred[:,1])
def plot_roc_curve(fpr, tpr,auc):
    plt.plot(fpr, tpr, color='orange', label="xgb_pca_lda_fs, auc=" + str(auc))
    plt.plot([0, 1], [0, 1], color='darkblue', linestyle='--')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver Operating Characteristic (ROC) Curve')
    plt.legend()
    plt.show()